In [3]:
import tensorflow as tf
import numpy as np

height,width=180,180

In [4]:
class_names = [ 'Corn___Common_rust', 
                'Corn___Gray_leaf_spot', 
                'Corn___Northern_Leaf_Blight', 
                'Corn___healthy', 
                'Potato___Early_blight', 
                'Potato___Late_blight', 
                'Potato___healthy', 
                'Strawberry___Leaf_scorch', 
                'Strawberry___healthy', 
                'Tomato___Early_blight', 
                'Tomato___Late_blight',
                'Tomato___Target_Spot', 
                'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 
                 'Tomato___healthy']

In [5]:
from keras.preprocessing import image
# 로컬 이미지 경로 설정
local_image_path = "./test-case/healthy_strawberry.jpg"

# 이미지 불러오기
img = image.load_img(local_image_path, target_size=(height, width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가

In [6]:
import cv2


PATH_TO_MODEL = './tensorlfow-lite-models/model-231025 (resnet).tflite'

interpreter = tf.lite.Interpreter(model_path=PATH_TO_MODEL)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[4.2313206e-07 5.5478484e-09 2.6668298e-10 9.2318624e-07 2.5046290e-10
  3.1400529e-10 1.0016653e-09 4.4239905e-09 1.8012155e-07 6.1336541e-06
  9.9999189e-01 1.7800609e-07 1.3943611e-07 8.6042910e-08]]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
interpreter.get_signature_list()

In [7]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

predictions_lite = classify_lite(resnet50_input = img_array)['dense_1']
score_lite = tf.nn.softmax(predictions_lite)

# 클래스 이름과 신뢰도 출력
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(predictions_lite)], 100 * np.max(predictions_lite))
)

This image most likely belongs to Strawberry___healthy with a 99.30 percent confidence.
